In [59]:
import whisper
import moviepy.editor as mp
from moviepy.video.tools.subtitles import SubtitlesClip
import spacy
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import re

In [60]:
def compute_lps_array(pattern):
    lps = [0] * len(pattern)
    length = 0
    i = 1
    while i < len(pattern):
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1
    return lps

def kmp_search(sentence, words_list):
    # Preprocess sentence into list of words
    pattern = sentence.lower().split()
    # print(words_list)
    text = [word["word"].lower() for word in words_list]

    # Compute LPS array
    lps = compute_lps_array(pattern)

    i = 0  # index for text
    j = 0  # index for pattern

    while i < len(text):
        if pattern[j] == text[i]:
            i += 1
            j += 1

        if j == len(pattern):
            # print(f"Found pattern at index {i - j}")
            j = lps[j - 1]
            return i - len(pattern)  # Return the starting index of the match
        elif i < len(text) and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1

    return -1  # If no match is found

In [61]:
def match_sentences_with_words(sentences, words):
    """Find sentences in the list of words using KMP algorithm."""
    results = {}
    for sentence in sentences:
        # sentence_words = sentence.split()
        matches = kmp_search(sentence.strip(), words)
        results[sentence.strip()] = matches
    return results

In [62]:
def transcribe_video(video_path):
    model = whisper.load_model("small.en")
    result = model.transcribe(video_path, word_timestamps=True)
    return result

In [63]:
def is_gif_worthy(text, nlp, sia):
    # Parse the text
    # print(text)
    doc = nlp(text)
    
    # Check for named entities (people, places, organizations)
    has_named_entity = any(ent.label_ in ['PERSON', 'ORG', 'GPE'] for ent in doc.ents)
    
    # Check for action verbs
    has_action_verb = any(token.pos_ == 'VERB' and token.dep_ == 'ROOT' for token in doc)
    
    # Analyze sentiment
    sentiment_scores = sia.polarity_scores(text)
    is_emotional = abs(sentiment_scores['compound']) > 0.2
    
    # Check for questions or exclamations
    is_question_or_exclamation = any(sent.text.strip().endswith(('?', '!')) for sent in doc.sents)
    
    # print((has_named_entity or has_action_verb or is_emotional or is_question_or_exclamation) and len(text.split()) <= 6)
    # Combine criteria
    return (has_named_entity or has_action_verb or is_emotional or is_question_or_exclamation) and len(text.split()) <= 6


In [70]:
def select_suitable_dialogues(segments, nlp, sia):
    suitable_sentences = []
    for segment in segments:
        if is_gif_worthy(segment, nlp, sia):
            suitable_sentences.append(segment)
    return suitable_sentences

def create_gif_with_caption(video_path, start_time, end_time, text, output_path):
    video = mp.VideoFileClip(video_path).subclip(start_time, end_time)
    
    # Create caption
    txt_clip = mp.TextClip(text, fontsize=128, color='white', font='Arial', stroke_color='black', stroke_width=2)
    txt_clip = txt_clip.set_pos('bottom').set_duration(video.duration)
    
    # Combine video and caption
    final_clip = mp.CompositeVideoClip([video, txt_clip])
    
    # Write GIF
    final_clip.write_gif(output_path)

In [65]:
nlp = spacy.load("en_core_web_sm")
video_path = 'long-vid.mp4'
sia = SentimentIntensityAnalyzer()

# Step 1: Transcribe the video
result = transcribe_video(video_path)


In [66]:
sentences = result["text"].split(".")
suitable_sentences = select_suitable_dialogues(sentences, nlp, sia)

words = []
for segment in result["segments"]:
    for word in segment['words']:
        if(word['word'][-1] == '.'):
            word['word'] = word['word'][:-1]
        words.append({'word': word['word'].strip(), 'start': word['start'], 'end': word['end']})

In [31]:
print(words)

[{'word': "What's", 'start': 0.7800000000000031, 'end': 1.26}, {'word': 'up?', 'start': 1.26, 'end': 1.48}, {'word': "I'm", 'start': 4.0, 'end': 4.48}, {'word': 'looking', 'start': 4.48, 'end': 4.62}, {'word': 'to', 'start': 4.62, 'end': 4.82}, {'word': 'hire', 'start': 4.82, 'end': 4.9}, {'word': 'someone', 'start': 4.9, 'end': 5.34}, {'word': 'so', 'start': 5.34, 'end': 6.16}, {'word': "I've", 'start': 6.16, 'end': 7.84}, {'word': 'got', 'start': 7.84, 'end': 7.96}, {'word': 'a', 'start': 7.96, 'end': 8.14}, {'word': 'cool', 'start': 8.14, 'end': 8.44}, {'word': 'gifty', 'start': 8.44, 'end': 8.86}, {'word': 'ranking', 'start': 8.86, 'end': 9.14}, {'word': 'system', 'start': 9.14, 'end': 9.78}, {'word': 'This', 'start': 10.68, 'end': 10.8}, {'word': 'gif', 'start': 10.8, 'end': 11.0}, {'word': 'of', 'start': 11.0, 'end': 11.2}, {'word': 'me', 'start': 11.2, 'end': 11.38}, {'word': 'has', 'start': 11.38, 'end': 11.62}, {'word': 'seven', 'start': 11.62, 'end': 11.88}, {'word': 'million

In [67]:
sentence_matches = match_sentences_with_words(suitable_sentences, words)

In [68]:
suitable_segments = []
for sentence, match in sentence_matches.items():
    if match != -1:
        start_time = words[match]['start']
        end_time = words[match + len(sentence.split()) - 1]['end']
        suitable_segments.append({"text": sentence, "start": start_time, "end": end_time})


In [71]:
# Step 3: Create GIFs with captions
for i, segment in enumerate(suitable_segments):
    start_time = segment["start"]
    end_time = segment["end"]
    text = segment["text"]
    
    # Clean up text (remove newlines, extra spaces)
    text = re.sub(r'\s+', ' ', text).strip()
    
    output_path = f"output_gif_{i}.gif"
    create_gif_with_caption(video_path, start_time, end_time, text, output_path)
    print(f"Created GIF: {output_path}")

MoviePy - Building file output_gif_0.gif with imageio.


Created GIF: output_gif_0.gif
MoviePy - Building file output_gif_1.gif with imageio.


Created GIF: output_gif_1.gif
MoviePy - Building file output_gif_2.gif with imageio.


Created GIF: output_gif_2.gif
MoviePy - Building file output_gif_3.gif with imageio.


Created GIF: output_gif_3.gif
MoviePy - Building file output_gif_4.gif with imageio.


...

Created GIF: output_gif_11.gif
MoviePy - Building file output_gif_12.gif with imageio.


Created GIF: output_gif_12.gif
MoviePy - Building file output_gif_13.gif with imageio.


Created GIF: output_gif_13.gif
MoviePy - Building file output_gif_14.gif with imageio.


Created GIF: output_gif_14.gif
MoviePy - Building file output_gif_15.gif with imageio.


Created GIF: output_gif_15.gif
